In [ ]:
import sys, time, os, asyncio, glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from astropy.time import Time, TimeDelta
from lsst_efd_client import EfdClient

In [ ]:
client = EfdClient('summit_efd')

In [ ]:
dates = ["2023-07-04","2023-07-05","2023-07-06",\
         "2023-07-18","2023-07-20","2023-08-01","2023-08-02","2023-08-03","2023-08-15","2023-08-16","2023-08-17",\
         "2023-08-29","2023-09-12","2023-09-13","2023-09-14","2023-09-26","2023-09-27","2023-09-28"]
closes = []
winds = []
for date in dates:
    start = Time(f"{date}T16:00:00Z", scale='utc')
    date_split = date.split('-')
    end_date = f"{date_split[0]}-{date_split[1]}-{int(date_split[2])+1:02}"
    print(date, end_date)
    end = Time(f"{end_date}T10:00:00Z", scale='utc')
    try:
        state = await client.select_time_series('lsst.sal.ATDome.logevent_mainDoorState', \
                                            ['state'],  start, end)
        wind = await client.select_time_series('lsst.sal.ESS.airFlow', \
                                                    ['speed'],  start, end)
        print(len(state), len(wind))
        close = state[state['state'] == 5]
        windMed = np.median(wind.values)
        closes.append(len(close))
        winds.append(windMed)
    except:
        continue


In [ ]:
fig, ax = plt.subplots(1,1, figsize=(5,5))
ax.set_title(f"Phantom Closes vs Wind Speed", fontsize=16)
ax.scatter(winds, closes)
ax.set_ylabel("Number of close events per night")
ax.set_xlabel("Median wind speed (m/s)")
plt.savefig("/home/craiglagegit/DATA/Shutter_Closes_vs_Wind_Speed_29Sep23.png")
